Optimisation de l'entrainement pour `focus` 
This is the same function as used in `10_Transfer_learning_what_networks.ipynb`
> ... TODO ... # TODO test without circular padding, with Adam, with no warmstart 

    model = torchvision.models.resnet18(weights=None)

# optimize meta-parameters

In [1]:
import dataclasses

In [2]:
%ls -ltr cached_data/34*

-rw-r--r--@ 1 laurentperrinet  staff  122880 28 nov.  12:03 cached_data/34_optuna_resnet.sqlite3
-rw-r--r--  1 laurentperrinet  staff  204800 30 nov.  07:51 cached_data/34_optuna_bis.sqlite3
-rw-r--r--@ 1 laurentperrinet  staff  163840  1 déc.  10:52 cached_data/34_optuna_convnext.sqlite3
-rw-r--r--  1 laurentperrinet  staff  200704  4 déc.  08:06 cached_data/34_optuna_2025-12-04.sqlite3
-rw-r--r--  1 laurentperrinet  staff  286720  6 déc.  03:34 cached_data/34_optuna_samedi.sqlite3
-rw-r--r--  1 laurentperrinet  staff  290816  7 déc.  12:36 cached_data/34_optuna.sqlite3


In [3]:
# print_gpu_memory()

# print(path_save)
# %ls -l {path}*

# %rm "cached_data/34_optuna.sqlite3"  # FORCING RECOMPUTE

In [4]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [5]:
# import retinoto_py as fovea
# batch_size = 64
# args = fovea.Params()
# opts_dict = dict(do_fovea=True, num_epochs=1, verbose=False)
# args = dataclasses.replace(args, **opts_dict)
# args

In [6]:
import retinoto_py as fovea
args = fovea.Params(do_fovea=True, num_epochs=1, subset_factor=10, verbose=False)
args

Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-7.5, padding_mode='zeros', seed=2025, batch_size=80, num_workers=1, in_memory=False, model_name='convnext_base', num_epochs=1, subset_factor=10, lr=2e-07, delta1=0.2, delta2=0.007, weight_decay=0.003, label_smoothing=0.05, do_full_training=False, shuffle=True, verbose=False)

In [7]:
%ls -ltr cached_data/32*

-rw-r--r--  1 laurentperrinet  staff  354501345  3 déc.  15:39 cached_data/32_fovea_model_name=convnext_base_dataset=bbox_20epochs.pth
-rw-r--r--  1 laurentperrinet  staff       3811  5 déc.  11:45 cached_data/32_fovea_model_name=convnext_base_dataset=bbox_20epochs.json
-rw-r--r--  1 laurentperrinet  staff  354501345  6 déc.  07:25 cached_data/32_fovea_model_name=convnext_base_dataset=bbox_40epochs.pth
-rw-r--r--  1 laurentperrinet  staff       7597  6 déc.  07:25 cached_data/32_fovea_model_name=convnext_base_dataset=bbox_40epochs.json
-rw-r--r--  1 laurentperrinet  staff          0  8 déc.  00:15 cached_data/32_fovea_model_name=convnext_base_dataset=bbox.lock
-rw-r--r--  1 laurentperrinet  staff  354501345  8 déc.  00:53 cached_data/32_fovea_model_name=convnext_base_dataset=bbox.pth
-rw-r--r--  1 laurentperrinet  staff       7785  8 déc.  00:53 cached_data/32_fovea_model_name=convnext_base_dataset=bbox.json


In [8]:
%ls -ltr cached_data/33*

-rw-r--r--  1 laurentperrinet  staff  224  6 déc.  18:53 cached_data/33_model_comparison.json


In [9]:
dataset = 'bbox'
TRAIN_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'train'
train_dataset = fovea.get_dataset(args, TRAIN_DATA_DIR)
train_loader = fovea.get_loader(args, train_dataset)
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR)
val_loader = fovea.get_loader(args, val_dataset)

In [10]:
# model_filename = None # do not use resumed net
# model_filename = args.data_cache / f'32_fovea_model_name={args.model_name}_dataset={dataset}_retrained_bkp.pth'
# model_filename = args.data_cache / f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model_filename = args.data_cache / f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

PosixPath('cached_data/32_fovea_model_name=convnext_base_dataset=bbox.pth')

In [ ]:
accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Evaluating {args.model_name} on dataset: {dataset}")


Evaluating convnext_base on dataset: bbox:   0%|          | 0/62 [00:03<?, ?it/s]

In [ ]:
print(f'{accuracy=}')

In [ ]:
if fovea.torch.cuda.is_available(): fovea.torch.cuda.empty_cache()

In [ ]:
args = fovea.Params(do_fovea=True, num_epochs=1, 
                    batch_size=16, # HACK for CONEC-LID-002
                    verbose=False)
def objective(trial):
    model = fovea.load_model(args, model_filename=model_filename)

    # args.batch_size = trial.suggest_int('batch_size', 16, 512, log=True, step=1)
    args.rs_min = trial.suggest_float( 'rs_min', -2, 0.1)
    args.rs_max = trial.suggest_float( 'rs_max', -8, -4)
    
    scale = 10
    if args.delta2>0: args.delta2 = trial.suggest_float('delta2', args.delta2 / scale, min((args.delta2 * scale, 1)), log=True)
    args.lr = trial.suggest_float('lr', args.lr / scale, args.lr * scale, log=True)

    args.delta1 = trial.suggest_float('delta1', args.delta1 / scale, min((args.delta1 * scale, 1)), log=True)
    if args.weight_decay>0: args.weight_decay = trial.suggest_float('weight_decay', args.weight_decay / scale, args.weight_decay * scale, log=True)
    if args.label_smoothing>0: args.label_smoothing = trial.suggest_float('label_smoothing', args.label_smoothing / scale, args.label_smoothing * scale, log=True)

    scale = 100 

    # args.im_mean = trial.suggest_float('im_mean', opt.im_mean / scale, opt.im_mean * scale, log=True)
    # args.im_std = trial.suggest_float('im_std', opt.im_std / scale, opt.im_std * scale, log=True)

    # get the architecture of the network, train and get accuracy on the validation set
    train_dataset.transform = train_dataset.transform = fovea.get_preprocess(args) 
    train_loader = fovea.get_loader(args, train_dataset)
    val_loader = fovea.get_loader(args, val_dataset)
    _, df_train = fovea.train_model(args, model=model, train_loader=train_loader, val_loader=val_loader)
    accuracy = df_train['acc_val'].mean()
    
    return accuracy
    
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(storage=f"sqlite:///cached_data/34_optuna.sqlite3", 
                            sampler=optuna.samplers.TPESampler(multivariate=False, warn_independent_sampling=False),
                            direction='maximize', load_if_exists=True, study_name=f"34_optuna")
print(f'Starting optimization with params: {args} on {max(150-len(study.trials), 0)} trials - {len(study.trials)} ')
study.optimize(objective, n_trials=max((150-len(study.trials), 0)), n_jobs=1, show_progress_bar=True)

print(50*'-.')
print("Best params: ", study.best_params)
print("Best value: ", study.best_value)
print("Best Trial: ", study.best_trial)
# print("Trials: ", study.trials)

In [ ]:
import optuna.visualization.matplotlib as vis

In [ ]:
vis.plot_param_importances(study)

In [ ]:
# df = study.trials_dataframe()   
params = sorted({k for t in study.trials for k in t.params})
params

In [ ]:
import seaborn as sns
fig, axes = fovea.plt.subplots(len(params), 1, figsize=(15, 8*len(params)), sharey=True)

for ax, pname in zip(axes, params):
    xs = [t.params[pname] for t in study.trials if pname in t.params]
    ys = [t.value for t in study.trials if pname in t.params]

    sns.kdeplot(
        x=xs,
        y=ys,
        ax=ax,
        color="red",
        bw_adjust=0.5,  # Bandwidth adjustment (smoothing level)
        log_transform=(True, False),  # Log transform only the x-axis
    )

    ax.scatter(xs, ys, s=20, alpha=1., color='blue')

    ax.set_xlabel(pname)
    ax.set_xscale('log')
    ax.set_ylabel("Objective")

fovea.plt.tight_layout()

In [ ]:
# vis.plot_contour(study, params=["weight_quanta", "resample_rate"])
# vis.plot_contour(study, params=["num_particles", "chunk_size"])